In [5]:
import pandas as pd

In [40]:
sleep = pd.read_csv('../data/total_sleep_hours.csv', index_col='date', parse_dates=['date'])

In [41]:
# Clean up data
sleep = sleep.resample('D').sum().fillna('0')
sleep = sleep.replace(0, sleep.mean())

In [42]:
sleep.describe()

,total_sleep_hours
count,833.000000
mean,7.030128
std,1.743230
min,1.500000
25%,6.000000
50%,6.980000
75%,8.000000
max,16.000000


In [43]:
pd.to_datetime(sleep[sleep['total_sleep_hours'] < 3].index)

DatetimeIndex(['2021-03-18', '2021-08-08', '2021-08-22', '2021-12-18',
               '2022-07-15'],
              dtype='datetime64[ns]', name='date', freq=None)